In [1]:
import torch
import train
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import optim

import dataset
import model
import utils

c:\Users\USER\anaconda3\envs\SepehrEslami\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
info = {'num_epoch' : 1,
        'device' : 'cuda',
        'reset': True,
        'name_load' : 'model_loss0.5932',
        'model_load_path' : './model/',
        'model_save_path' : './model/MI'
        }

In [3]:
loss_train_hist = []
loss_valid_hist = []

metric_train_hist = []
metric_valid_hist = []

In [4]:
load_path = info['model_load_path'] + info['name_load'] + ".pth"

In [5]:
# model = model.Resnet(120, reset=info['reset'])
model = model.MultiInputModel(1)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.001)  #  optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)

In [6]:
model, loss_fn, optimizer = train.load(model, loss_fn, optimizer, device=info['device'], reset = info['reset'], load_path = load_path)

train_loader = dataset.UTKFace(mode='train')(batch_size=32)
valid_loader = dataset.UTKFace(mode='valid')(batch_size=124)
# train_loader = dataset.CACD(train=True)(batch_size=124)
# valid_loader = dataset.CACD(train=False)(batch_size=264)

In [2]:
print(torch.cuda.is_available())

False


In [15]:
epochs = info['num_epoch']

for epoch in range(epochs):
    _, loss_train, metric_train = utils.train_one_epoch(model, train_loader, loss_fn, optimizer, epoch=epoch, device='cuda')
    loss_valid, metric_valid = utils.evaluate(model, valid_loader, loss_fn, device='cuda')

        
    loss_train_hist.append(loss_train)
    loss_valid_hist.append(loss_valid)

    metric_train_hist.append(metric_train)
    metric_valid_hist.append(metric_valid)


    print(f'Train      - Loss:{loss_train}  Metric:{metric_train}')
    print(f'Validation - Loss:{loss_valid}  Metric:{metric_valid}')
    print()

save_path = info['model_save_path'] + 'model_loss' +f'{loss_train:.4}'+ ".pth"
train.save(save_path, model, optimizer, loss_fn)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
train.plot(metric_train_hist, metric_valid_hist, "Metric")

In [ ]:
train.plot(loss_train_hist, loss_valid_hist, 'Loss')